# 爬PTT地震文

* 需要的資訊：
    
    1. 發文時間
    2. 文章標題
    3. 文章內容
    4. 前100推推文內容
    5. 發文者ID
    6. 發文者IP位址
    7. 該文是否屬於地震文（dummy）
    8. 該文是否搶到地震爆文（dummy）
   
* 另外要寫的功能：

    1. 用IP位址反查地址資訊，回傳「縣市」（str）及經緯度（某種可以被餵進google map API的格式）
    2. 一天／一週當中PTT上站人數的pattern，輔以「當時發文者發文所處的時間」來控制error term

In [1]:
# -*- coding: utf-8 -*-

# 導入 模組(module) 
import requests 
# 導入 BeautifulSoup 模組(module)：解析HTML 語法工具
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

from multiprocessing import Pool #https://github.com/leVirve/CrawlerTutorial

/Applications/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#八卦版文章連結
url = "https://www.ptt.cc/bbs/Gossiping/index.html"
header_over18 = {"cookie":"over18=1"}
res = requests.get(url, headers = header_over18)
res

<Response [200]>

In [3]:
soup = BeautifulSoup(res.text, "html.parser")

# try to search keyword "地震"

In [4]:
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
resp = requests.get(search_endpoint_url, params={'q': '震'}, headers = header_over18)

In [5]:
soup = BeautifulSoup(resp.text, "html.parser")

In [6]:
# 找到全部地震文章的連結
links = soup.find_all("a href")

In [7]:
titles = soup.find_all("div", "title")

In [8]:
for title in titles:
    print(title.text, title.a["href"])
    link = "https://www.ptt.cc/" + title.a["href"]


[問卦] 地震搖晃程度和屋齡有關嗎？
 /bbs/Gossiping/M.1592355119.A.397.html

[爆卦] 地震
 /bbs/Gossiping/M.1592349530.A.F00.html

[爆卦] 地震
 /bbs/Gossiping/M.1592349499.A.FF3.html

[爆卦] 地震
 /bbs/Gossiping/M.1592349494.A.959.html

地震
 /bbs/Gossiping/M.1592349492.A.549.html

[爆卦] 地震
 /bbs/Gossiping/M.1592349487.A.636.html

地震
 /bbs/Gossiping/M.1592349482.A.D1B.html

[爆卦] 地震!
 /bbs/Gossiping/M.1592349476.A.46C.html

[爆卦] 地震
 /bbs/Gossiping/M.1592349474.A.EC6.html

地震
 /bbs/Gossiping/M.1592349473.A.B7A.html

[爆卦] 地震
 /bbs/Gossiping/M.1592349459.A.902.html

Re: [問卦] 功高震主時，要怎麼樣給主管台階下
 /bbs/Gossiping/M.1592330099.A.335.html

[問卦] 功高震主時，要怎麼樣給主管台階下
 /bbs/Gossiping/M.1592329647.A.976.html

[新聞] 陳時中功高震主？被蔡英文、蘇貞昌圍剿？
 /bbs/Gossiping/M.1592295120.A.0EE.html

[問卦] 華人演包青天把臉塗黑 黑人會震怒嗎
 /bbs/Gossiping/M.1592229107.A.383.html

[問卦] 日本友人震驚：台灣竟是炒房天堂？
 /bbs/Gossiping/M.1592197082.A.D91.html

地震
 /bbs/Gossiping/M.1592158410.A.D2E.html

[爆卦] 地震
 /bbs/Gossiping/M.1592158328.A.3AC.html

[爆卦]地震 阿芳我愛妳
 /bbs/Gossiping/M.1592158280.A.80C

In [9]:
article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
article_id

'1592146308'

In [10]:
# 進入搜尋頁面的第一篇文章
link = "https://www.ptt.cc/" + title.a["href"]

# 18禁
payload = {
    "from" : "/bbs/Gossiping/index.html",
    "yes"  : "yes"
}
rs = requests.session()
res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)

res_subpage = rs.get(link, verify = False)
soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

/Applications/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Applications/anaconda

In [11]:
print(soup_subpage.text)






[問卦] 今天早上的地震很大嗎? - 看板 Gossiping - 批踢踢實業坊
















批踢踢實業坊
›
看板 Gossiping
關於我們
聯絡資訊




返回看板




作者wurara22 (烏拉餓餓)看板Gossiping標題[問卦] 今天早上的地震很大嗎?時間Sun Jun 14 22:51:41 2020
大家晚安
如題

剛剛看新聞才看到今天早上4:19有地震
還說是全台有感 規模6.0

可是魯妹根本沒感覺...

之前半夜有地震的話我都會醒來


昨晚跟平常不同的是有戴耳塞睡覺
可能因為耳塞沒聽到家具震動的聲音所以沒注意到地震??

有沒有八卦?0'_'0

--
——————
咚踏取蜜！
https://i.imgur.com/ymESrcv.jpg
(●’ω`●）ノ dustycat_udon
——————

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 118.166.126.3 (臺灣)
※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1592146308.A.988.html
→ akila08539: 灰塵貓有感覺嗎 06/14 22:52
今天早上的因為我沒感覺所以也不知道灰塵貓怎樣
不過之前有地震的時候她是沒什麼反應，頂多抬頭四處觀望
學姐貓是完全睡死zzz

我是有在床頭櫃放一個洗衣袋啦，有需要把她包進去一起逃
噓 jenn8588: 沒人關心 06/14 22:52
推 XXXXLINDA: 還好 06/14 22:52
→ saobox: 意淫地震 8888888 06/14 22:52
推 purplebfly: 事實上我是被手機吵醒,不是被地震吵醒的 06/14 22:52
這也很奇怪
之前我會收到警報，但是今天早上沒有
推 wade6510: 最大三級  因為在海裡 算中層深度 06/14 22:52
※ 編輯: wurara22 (118.166.126.3 臺灣), 06/14/2020 22:56:08
推 Xhocer: 我睡得跟死豬依樣 06/14 22:52
推 dWoWb: 是喔...早上起來看八卦版也沒特別看到地震 LUL 06/14 22:54
因為新聞寫是今年最強，但是我卻睡到翻，所以想知道到底是不是今年最強


In [12]:
#一篇文章的 作者、標題、時間、看板
header = soup_subpage.find_all("span", "article-meta-value")
print(header)

[<span class="article-meta-value">wurara22 (烏拉餓餓)</span>, <span class="article-meta-value">Gossiping</span>, <span class="article-meta-value">[問卦] 今天早上的地震很大嗎?</span>, <span class="article-meta-value">Sun Jun 14 22:51:41 2020</span>]


In [13]:
print(header[0].text) #author
print(header[1].text) #board
print(header[2].text) #title
print(header[3].text) #date

wurara22 (烏拉餓餓)
Gossiping
[問卦] 今天早上的地震很大嗎?
Sun Jun 14 22:51:41 2020


In [14]:
main_text = soup_subpage.find(id = "main-container").text
print(main_text)


作者wurara22 (烏拉餓餓)看板Gossiping標題[問卦] 今天早上的地震很大嗎?時間Sun Jun 14 22:51:41 2020
大家晚安
如題

剛剛看新聞才看到今天早上4:19有地震
還說是全台有感 規模6.0

可是魯妹根本沒感覺...

之前半夜有地震的話我都會醒來


昨晚跟平常不同的是有戴耳塞睡覺
可能因為耳塞沒聽到家具震動的聲音所以沒注意到地震??

有沒有八卦?0'_'0

--
——————
咚踏取蜜！
https://i.imgur.com/ymESrcv.jpg
(●’ω`●）ノ dustycat_udon
——————

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 118.166.126.3 (臺灣)
※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1592146308.A.988.html
→ akila08539: 灰塵貓有感覺嗎 06/14 22:52
今天早上的因為我沒感覺所以也不知道灰塵貓怎樣
不過之前有地震的時候她是沒什麼反應，頂多抬頭四處觀望
學姐貓是完全睡死zzz

我是有在床頭櫃放一個洗衣袋啦，有需要把她包進去一起逃
噓 jenn8588: 沒人關心 06/14 22:52
推 XXXXLINDA: 還好 06/14 22:52
→ saobox: 意淫地震 8888888 06/14 22:52
推 purplebfly: 事實上我是被手機吵醒,不是被地震吵醒的 06/14 22:52
這也很奇怪
之前我會收到警報，但是今天早上沒有
推 wade6510: 最大三級  因為在海裡 算中層深度 06/14 22:52
※ 編輯: wurara22 (118.166.126.3 臺灣), 06/14/2020 22:56:08
推 Xhocer: 我睡得跟死豬依樣 06/14 22:52
推 dWoWb: 是喔...早上起來看八卦版也沒特別看到地震 LUL 06/14 22:54
因為新聞寫是今年最強，但是我卻睡到翻，所以想知道到底是不是今年最強
※ 編輯: wurara22 (118.166.126.3 臺灣), 06/14/2020 22:57:05
→ SFGiants: 美國無感 XD 06/14 23:01
推 ted01234567: 有感 但

In [15]:
#推文
main_text.split(sep = "html")[1].split(sep = "\n")[1:]

['→ akila08539: 灰塵貓有感覺嗎 06/14 22:52',
 '今天早上的因為我沒感覺所以也不知道灰塵貓怎樣',
 '不過之前有地震的時候她是沒什麼反應，頂多抬頭四處觀望',
 '學姐貓是完全睡死zzz',
 '',
 '我是有在床頭櫃放一個洗衣袋啦，有需要把她包進去一起逃',
 '噓 jenn8588: 沒人關心 06/14 22:52',
 '推 XXXXLINDA: 還好 06/14 22:52',
 '→ saobox: 意淫地震 8888888 06/14 22:52',
 '推 purplebfly: 事實上我是被手機吵醒,不是被地震吵醒的 06/14 22:52',
 '這也很奇怪',
 '之前我會收到警報，但是今天早上沒有',
 '推 wade6510: 最大三級  因為在海裡 算中層深度 06/14 22:52',
 '※ 編輯: wurara22 (118.166.126.3 臺灣), 06/14/2020 22:56:08',
 '推 Xhocer: 我睡得跟死豬依樣 06/14 22:52',
 '推 dWoWb: 是喔...早上起來看八卦版也沒特別看到地震 LUL 06/14 22:54',
 '因為新聞寫是今年最強，但是我卻睡到翻，所以想知道到底是不是今年最強',
 '※ 編輯: wurara22 (118.166.126.3 臺灣), 06/14/2020 22:57:05',
 '→ SFGiants: 美國無感 XD 06/14 23:01',
 '推 ted01234567: 有感 但是太累了繼續睡 06/14 23:43',
 '→ a3221715: 睡不好 不知道是不是因為地震 06/14 23:55',
 '→ a3221715: 但睡死無感 06/14 23:55',
 '推 cuteSquirrel: 很晃 06/15 00:05',
 '推 tuutu: 有感 我的鸚鵡被驚醒亂飛 後來我把他抓來一起睡 06/15 01:53',
 '',
 '',
 '本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。',
 '']

In [16]:
# 把整個內容切割透過 "-- " 切割成2個陣列
pre_text = main_text.split('--')[0]
    
# 把每段文字 根據 '\n' 切開
texts = pre_text.split('\n')
# 如果你爬多篇你會發現 
contents = texts[2:]
# 內容
content = '\n'.join(contents)

print('內容：'+content)

內容：大家晚安
如題

剛剛看新聞才看到今天早上4:19有地震
還說是全台有感 規模6.0

可是魯妹根本沒感覺...

之前半夜有地震的話我都會醒來


昨晚跟平常不同的是有戴耳塞睡覺
可能因為耳塞沒聽到家具震動的聲音所以沒注意到地震??

有沒有八卦?0'_'0




In [17]:
IP_address = main_text.split('來自:')[1].split("\n")[0]
IP_address

' 118.166.126.3 (臺灣)'

```
f"{article_id}" = {
    "Date" : ""
    "Title" : ""
    "Author" : ""
    "IP" : ""
    "Content" : 
}
```

In [18]:
f"{article_id}"

'1592146308'

In [19]:
exec_str = "M" + f"{article_id}"
exec_str

'M1592146308'

In [20]:
exec(exec_str+" = {}", globals())

In [21]:
type(M1590366890)

NameError: name 'M1590366890' is not defined

In [22]:
M1590366890[1] = "01"
M1590366890[2] = "02"

NameError: name 'M1590366890' is not defined

In [23]:
exec(f"{exec_str}", globals())

## 實作：把搜尋頁第一頁的文章都爬下來，存在一個dict內

In [24]:
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
#search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
#resp = requests.get(search_endpoint_url, params={'q': '地震'}, headers = header_over18)

#soup = BeautifulSoup(resp.text, "html.parser")

#忽略warning
import warnings
warnings.filterwarnings('ignore')

# 建立字典
earthquake_dict = {} # key是文章的link，value是另一本以文章的unique link為名的dict
# list內則有：發文時間、文章標題、作者ID、作者IP、文章內容
names = globals()

# 搜尋頁面換頁
# https://www.ptt.cc/bbs/Gossiping/search?page= 3 &q=%E5%9C%B0%E9%9C%87
# key換成index
index = 1
for page_count in range(1, 52+1):
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=%E5%9C%B0%E9%9C%87"
    search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=震"
    resp = requests.get(search_page_url, headers = header_over18)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # 找到該搜尋頁面下 全部地震文章的連結
    titles = soup.find_all("div", "title") #找到每篇文章的標題及連結
    for title in titles:
        #print(title.text, title.a["href"])
        
        link = "https://www.ptt.cc/" + title.a["href"]
        article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
        article_id = index
        index+=1
        
        #為這篇文章建一個dictionary
        name_str = "M" + f"{article_id}"
        names["M_%s" %article_id] = {}
        earthquake_dict[name_str] = names["M_%s" %article_id]

        
        # 進入搜尋頁面的第一篇文章
        payload = {
            "from" : "/bbs/Gossiping/index.html",
            "yes"  : "yes"
        }
        rs = requests.session()
        res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)
        res_subpage = rs.get(link, verify = False)
        soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

        #一篇文章的 作者、標題、時間、看板
        header = soup_subpage.find_all("span", "article-meta-value")
        #print(header)
        #print(header[0].text) #author
        #print(header[1].text) #board
        #print(header[2].text) #title
        #print(header[3].text) #date
        
        try: #如果有「看板」的話
            names["M_%s" %article_id]["Author"] = header[0].text
            names["M_%s" %article_id]["Board"] = header[1].text
            names["M_%s" %article_id]["Title"] = header[2].text
            names["M_%s" %article_id]["Date"] = header[3].text
        except:
            try: #如果是回覆文章，沒有「看板」的話
                names["M_%s" %article_id]["Author"] = header[0].text
                names["M_%s" %article_id]["Title"] = header[1].text
                names["M_%s" %article_id]["Date"] = header[2].text
            except:
                pass

        # 內文
        try:
            main_text = soup_subpage.find(id = "main-container").text
        except:
            pass
        #print(main_text)
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = main_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = '\n'.join(contents)
        #print('內容：'+content)
        names["M_%s" %article_id]["Content"] = content

        
        #IP位址
        try:
            try:
                IP_address = main_text.split('來自:')[1].split("\n")[0]
                #print(IP_address)
                names["M_%s" %article_id]["Full IP"] = IP_address
            except:
                try:
                    author = header[0].text
                    authorID = author.split(sep = " (")[0]
                    #print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])
                    IP_address = main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:]
                    names["M_%s" %article_id]["Full IP"] = IP_address
                except:
                    pass
        except:
            names["M_%s" %article_id]["Full IP"] = "Not Found"
        
        #只有數字的IP
        try:
            IP_shorten = IP_address.split(sep = " ")[1]
            names["M_%s" %article_id]["IP"] = IP_shorten
        except:
            pass
        
        # 推文
        try:            
            push = main_text.split(sep = "html")[1].split(sep = "\n")[1:]
            names["M_%s" %article_id]["Push"] = push
        except:
            names["M_%s" %article_id]["Push"] = "No Push"
        
        time.sleep(0.1)

# 八卦版前一百頁

In [175]:
# 八卦版前一百頁
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
#search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
#resp = requests.get(search_endpoint_url, params={'q': '地震'}, headers = header_over18)

#soup = BeautifulSoup(resp.text, "html.parser")

#忽略warning
import warnings
warnings.filterwarnings('ignore')

# 建立字典
earthquake_dict = {} # key是文章的link，value是另一本以文章的unique link為名的dict
# list內則有：發文時間、文章標題、作者ID、作者IP、文章內容
names = globals()

# 搜尋頁面換頁
# https://www.ptt.cc/bbs/Gossiping/search?page= 3 &q=%E5%9C%B0%E9%9C%87
# key換成index
index = 1
for page_count in range(39284, 39184, -1):
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=%E5%9C%B0%E9%9C%87"
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=震"
    search_page_url = "https://www.ptt.cc/bbs/Gossiping/index" + str(page_count) + ".html"
    resp = requests.get(search_page_url, headers = header_over18)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # 找到該搜尋頁面下 全部地震文章的連結
    titles = soup.find_all("div", "title") #找到每篇文章的標題及連結
    for title in titles:
        #print(title.text, title.a["href"])
        
        try:
            link = "https://www.ptt.cc/" + title.a["href"]
        except:
            pass
        #article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
        article_id = index
        index+=1
        
        #為這篇文章建一個dictionary
        name_str = "M" + f"{article_id}"
        names["M_%s" %article_id] = {}
        earthquake_dict[name_str] = names["M_%s" %article_id]

        
        # 進入搜尋頁面的第一篇文章
        payload = {
            "from" : "/bbs/Gossiping/index.html",
            "yes"  : "yes"
        }
        rs = requests.session()
        res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)
        res_subpage = rs.get(link, verify = False)
        soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

        #一篇文章的 作者、標題、時間、看板
        header = soup_subpage.find_all("span", "article-meta-value")
        #print(header)
        #print(header[0].text) #author
        #print(header[1].text) #board
        #print(header[2].text) #title
        #print(header[3].text) #date
        
        try: #如果有「看板」的話
            names["M_%s" %article_id]["Author"] = header[0].text
            names["M_%s" %article_id]["Board"] = header[1].text
            names["M_%s" %article_id]["Title"] = header[2].text
            names["M_%s" %article_id]["Date"] = header[3].text
        except:
            try: #如果是回覆文章，沒有「看板」的話
                names["M_%s" %article_id]["Author"] = header[0].text
                names["M_%s" %article_id]["Title"] = header[1].text
                names["M_%s" %article_id]["Date"] = header[2].text
            except:
                pass

        # 內文
        try:
            main_text = soup_subpage.find(id = "main-container").text
        except:
            pass
        #print(main_text)
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = main_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = '\n'.join(contents)
        #print('內容：'+content)
        names["M_%s" %article_id]["Content"] = content

        
        #IP位址
        try:
            try:
                IP_address = main_text.split('來自:')[1].split("\n")[0]
                #print(IP_address)
                names["M_%s" %article_id]["Full IP"] = IP_address
            except:
                try:
                    author = header[0].text
                    authorID = author.split(sep = " (")[0]
                    #print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])
                    IP_address = main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:]
                    names["M_%s" %article_id]["Full IP"] = IP_address
                except:
                    pass
        except:
            names["M_%s" %article_id]["Full IP"] = "Not Found"
        
        #只有數字的IP
        try:
            IP_shorten = IP_address.split(sep = " ")[1]
            names["M_%s" %article_id]["IP"] = IP_shorten
        except:
            pass
        
        # 推文
        try:            
            push = main_text.split(sep = "html")[1].split(sep = "\n")[1:]
            names["M_%s" %article_id]["Push"] = push
        except:
            names["M_%s" %article_id]["Push"] = "No Push"
        
        time.sleep(0.1)

In [25]:
title

<div class="title">
<a href="/bbs/Gossiping/M.1580085626.A.7F4.html">[問卦] 賈伯斯死掉跟kobe死掉哪個比較震撼?</a>
</div>

In [26]:
header[0]

<span class="article-meta-value">bluebluelan (白魚挺粗強貝柱連根立)</span>

In [27]:
author = header[0].text
authorID = author.split(sep = " (")[0]
#print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])

In [28]:
authorID

'bluebluelan'

In [29]:
page_count

52

In [30]:
len(earthquake_dict)

1040

In [31]:
earthquake_dict.keys()

dict_keys(['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23', 'M24', 'M25', 'M26', 'M27', 'M28', 'M29', 'M30', 'M31', 'M32', 'M33', 'M34', 'M35', 'M36', 'M37', 'M38', 'M39', 'M40', 'M41', 'M42', 'M43', 'M44', 'M45', 'M46', 'M47', 'M48', 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62', 'M63', 'M64', 'M65', 'M66', 'M67', 'M68', 'M69', 'M70', 'M71', 'M72', 'M73', 'M74', 'M75', 'M76', 'M77', 'M78', 'M79', 'M80', 'M81', 'M82', 'M83', 'M84', 'M85', 'M86', 'M87', 'M88', 'M89', 'M90', 'M91', 'M92', 'M93', 'M94', 'M95', 'M96', 'M97', 'M98', 'M99', 'M100', 'M101', 'M102', 'M103', 'M104', 'M105', 'M106', 'M107', 'M108', 'M109', 'M110', 'M111', 'M112', 'M113', 'M114', 'M115', 'M116', 'M117', 'M118', 'M119', 'M120', 'M121', 'M122', 'M123', 'M124', 'M125', 'M126', 'M127', 'M128', 'M129', 'M130', 'M131', 'M132', 'M133', 'M134', 'M135', 'M136', 'M137', '

In [32]:
earthquake_dict["M3"]

{'Author': 'musashi4735 (武藏)',
 'Board': 'Gossiping',
 'Title': '[爆卦] 地震',
 'Date': 'Wed Jun 17 07:18:17 2020',
 'Content': '地震!!中壢\n\n',
 'Full IP': ' 114.32.216.39 (臺灣)',
 'IP': '114.32.216.39',
 'Push': ['※ 編輯: musashi4735 (114.32.216.39 臺灣), 06/17/2020 07:18:51',
  '',
  '',
  '本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。',
  '']}

In [33]:
IP_shorten = IP_address.split(sep = " ")[1]
IP_shorten

'73.63.220.93'

# Save Dictionary

In [34]:
# Save
np.save("earthquake_dict_0617地震", earthquake_dict)

# Load
#read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()

In [35]:
earthquake_dict["M20"]

{'Author': 'wurara22 (烏拉餓餓)',
 'Board': 'Gossiping',
 'Title': '[問卦] 今天早上的地震很大嗎?',
 'Date': 'Sun Jun 14 22:51:41 2020',
 'Content': "大家晚安\n如題\n\n剛剛看新聞才看到今天早上4:19有地震\n還說是全台有感 規模6.0\n\n可是魯妹根本沒感覺...\n\n之前半夜有地震的話我都會醒來\n\n\n昨晚跟平常不同的是有戴耳塞睡覺\n可能因為耳塞沒聽到家具震動的聲音所以沒注意到地震??\n\n有沒有八卦?0'_'0\n\n",
 'Full IP': ' 118.166.126.3 (臺灣)',
 'IP': '118.166.126.3',
 'Push': ['→ akila08539: 灰塵貓有感覺嗎 06/14 22:52',
  '今天早上的因為我沒感覺所以也不知道灰塵貓怎樣',
  '不過之前有地震的時候她是沒什麼反應，頂多抬頭四處觀望',
  '學姐貓是完全睡死zzz',
  '',
  '我是有在床頭櫃放一個洗衣袋啦，有需要把她包進去一起逃',
  '噓 jenn8588: 沒人關心 06/14 22:52',
  '推 XXXXLINDA: 還好 06/14 22:52',
  '→ saobox: 意淫地震 8888888 06/14 22:52',
  '推 purplebfly: 事實上我是被手機吵醒,不是被地震吵醒的 06/14 22:52',
  '這也很奇怪',
  '之前我會收到警報，但是今天早上沒有',
  '推 wade6510: 最大三級  因為在海裡 算中層深度 06/14 22:52',
  '※ 編輯: wurara22 (118.166.126.3 臺灣), 06/14/2020 22:56:08',
  '推 Xhocer: 我睡得跟死豬依樣 06/14 22:52',
  '推 dWoWb: 是喔...早上起來看八卦版也沒特別看到地震 LUL 06/14 22:54',
  '因為新聞寫是今年最強，但是我卻睡到翻，所以想知道到底是不是今年最強',
  '※ 編輯: wurara22 (118.166.126.3 臺灣), 06/14/2020 22:57:05',
  '→ SFGi